In [24]:
import numpy as np
import pandas as pd
import os
import glob
import re
import matplotlib.pyplot as plt
import seaborn as sns
import re
from pathlib import Path
import datetime as dt

import warnings
warnings.filterwarnings("ignore")

# load in directories
from experiment_code.constants import Defaults
from experiment_code import preprocess
from experiment_code.targetfile_utils import Utils
from experiment_code.preprocess import PilotSentences

## Reviewed df stats

In [25]:
df = pd.read_csv(os.path.join(Defaults.RAW_DIR, f"cort_language_gorilla_v7.csv"))

In [26]:
def _get_response_type():
    response_type = "response_keyboard_single"
    return response_type

def _assign_trialtype(x):
    if x==False:
        value = "meaningful"
    elif x==True:
        value = "meaningless"
    else:
        value = x
    return value

def _rename_cols(df):
    """rename some columns for analysis
    """
    return df.rename({'Local Date':'local_date','Experiment ID':'experiment_id', 'Experiment Version':'experiment_version', 'Participant Public ID':'participant_public_id', 'Participant Private ID':'participant_id', 
                'Task Name':'task_name', 'Task Version':'task_version', 'Spreadsheet Name':'spreadsheet_version', 'Spreadsheet Row': 'spreadsheet_row', 'Trial Number': 'sentence_num', 'Zone Type':'zone_type', 
                'Reaction Time':'rt', 'Response':'response', 'Attempt':'attempt', 'Correct':'correct', 'Incorrect':'incorrect'}, axis=1)
    

# determine which cols to keep depending on task

df = df.filter({'Local Date', 'Experiment ID', 'Experiment Version', 'Participant Public ID', 'Participant Private ID',
        'Task Name', 'Task Version', 'Spreadsheet Name', 'Spreadsheet Row', 'Trial Number', 'Zone Type', 
        'Reaction Time', 'Response', 'Attempt', 'Correct', 'Incorrect', 'display', 'block_num', 'randomise_blocks',
        'full_sentence', 'last_word', 'sampled','CoRT_descript', 'CoRT_mean','condition_name',
        'CoRT_std','cloze_descript', 'cloze_probability', 'dataset', 'random_word', 'target_word', 'word_count'})

#rename some columns for analysis
df = _rename_cols(df)

# filter dataset to include trials and experimental blocks (i.e. not instructions)
response_type = _get_response_type() # response_type is different across the task
df = df.query(f'display=="trial" and block_num>0 and zone_type=="{response_type}"')
df['rt'] = df['rt'].astype(float)  

In [35]:
blocks = df.groupby('participant_id').apply(lambda x: x.sort_values('block_num').block_num).values
blocks

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 4., 4.,
        4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
        4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 

In [30]:
df['block_num'] = blocks

ValueError: Length of values does not match length of index

In [ ]:
# correct block_num to be sequential
df['block_num'] = df.apply(lambda x: x.sort_values('block_num').block_num).values
        
# get meaningful assesment
df['trial_type'] = df['sampled'].apply(lambda x: _assign_trialtype(x))

# get version
df["version"] = version

# get condition name (make sure it's just characters)
df['condition_name'] = df['condition_name'].str.extract('([a-zA-Z]*)')

df